In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, BatchNormalization, Conv2D, MaxPooling2D, Dropout
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import os
from PIL import Image

2023-03-31 06:58:55.594529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 06:58:56.442528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class BirdDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, use_rows, batch_size, csv_file="birds.csv"):
        self.csv_df = pd.read_csv(csv_file)
        self.use_rows = use_rows
        self.batch_size = batch_size 
        self.dataset = pd.DataFrame(self.csv_df).to_numpy()
    
        self.labels = self.dataset[:,2]
        self.img_filepaths = self.dataset[:,1]
        self.class_ids = self.dataset[:,0]
        
        (x,) = self.img_filepaths.shape
        print(self.dataset.shape)
        print(self.img_filepaths[0])
        image = Image.open(self.img_filepaths[0])
        image_array = np.expand_dims(np.asarray(image), axis=0)
        
#         print(image_array)
        print(image_array.shape)
        
#         self.images = []
        
#         for i in range(0, 10):
#             image = Image.open(self.img_filepaths[0])
#             image_array = np.expand_dims(np.asarray(image), axis=0)
#             self.images.append(image_array)
            
#         self.images = np.array(self.images)
        
#         print(self.images.shape)
    
    def __len__(self):
        # batches_per_epoch is the total number of batches used for one epoch
        batches_per_epoch = int(len(self.use_rows) / self.batch_size)
        return batches_per_epoch
    
    def __getitem__(self, index):
        # index is the index of the batch to be retrieved
        batch_ids = self.use_rows[index * self.batch_size: (index + 1) * self.batch_size]
        
        x = None
        y = None
        
        
        for curr_id in batch_ids:
            image = Image.open(self.img_filepaths[curr_id])
#             image = image.convert("L")
#             np_row = np.asarray(image)
            image_array = np.expand_dims(np.asarray(image),axis=0)
            
            if x is None:
                x = image_array
                y = self.class_ids[curr_id]
            else:
                x = np.vstack((x, image_array))
                y = np.vstack((y, self.class_ids[curr_id]))
                
    
#             print(self.class_ids[curr_id-1])
            
        
#         images = np.array(images)
#         id_labels = np.array(id_labels)
#         print(id_labels)
#         print(images.shape)
#         print(id_labels.shape)
#         print(self.class_ids)
        
#         print(self.class_ids.shape)
        
        
        y = tf.keras.utils.to_categorical(y, num_classes=511)
        print(x.shape)
        print(y.shape)
        return x, y            
            
#     def on_epoch_end(self):
#         np.random.shuffle(self.split_ids)
            
            

In [3]:
use_rows = list(range(0,87050))
bird_data_generator = BirdDataGenerator(csv_file="birds.csv", use_rows=use_rows, batch_size=128)
bird_data_generator.__getitem__(0)

(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)
(128, 224, 224, 3)
(128, 511)


(array([[[[180, 172, 149],
          [186, 178, 155],
          [186, 180, 158],
          ...,
          [181, 180, 198],
          [179, 180, 200],
          [178, 179, 199]],
 
         [[182, 174, 153],
          [190, 182, 161],
          [190, 184, 162],
          ...,
          [182, 181, 199],
          [176, 177, 197],
          [178, 179, 199]],
 
         [[186, 178, 159],
          [195, 187, 168],
          [196, 189, 170],
          ...,
          [184, 183, 199],
          [180, 179, 197],
          [183, 182, 200]],
 
         ...,
 
         [[151, 138, 121],
          [172, 159, 142],
          [178, 165, 146],
          ...,
          [189, 190, 211],
          [191, 189, 210],
          [169, 167, 188]],
 
         [[156, 140, 124],
          [155, 139, 123],
          [161, 148, 131],
          ...,
          [186, 188, 211],
          [182, 185, 204],
          [168, 171, 188]],
 
         [[157, 141, 125],
          [156, 140, 124],
          [167, 154, 137],
   

In [4]:
def bird_cnn_model(csv_file):
    model = Sequential([
        InputLayer(input_shape=(224,224,3)),
        Conv2D(filters=32, kernel_size=3, activation='relu'),
#         BatchNormalization(),
        MaxPooling2D(pool_size=2),
        Flatten(),
        Dense(511, activation="softmax")
    ])
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])
    
    print(model.summary())
    train_rows = list(range(0,50000))
    train_generator = BirdDataGenerator(csv_file=csv_file, use_rows=train_rows, batch_size=128)
    h = model.fit(x=train_generator, epochs=1, verbose=1)
    print(h)
    
#     validation_rows = list(range(84500, 87050))
#     validation_generator = BirdDataGenerator(csv_file=csv_file, use_rows=validation_rows, batch_size=128)
    
#     test_rows = list(range(81950, 84500))
#     test_generator = BirdDataGenerator(csv_file=csv_file, use_rows=test_rows, batch_size=128)
#     model.evaluate(x=test_generator)
    

In [5]:
cnn_model = bird_cnn_model("birds.csv")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 511)               201473503 
                                                                 
Total params: 201,474,399
Trainable params: 201,474,399
Non-trainable params: 0
_________________________________________________________________


2023-03-31 06:58:59.381582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 06:58:59.426587: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 06:58:59.427049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 06:58:59.429442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 06:58:59.429940: I tensorflow/compile

None
(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)
(128, 224, 224, 3)
(128, 511)


2023-03-31 06:59:02.779946: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


(128, 224, 224, 3)
(128, 511)
(128, 224, 224, 3)
(128, 511)


...

 70/390 [====>.........................] - ETA: 6:06 - loss: 648.3685 - accuracy: 0.0052(128, 224, 224, 3)
(128, 511)
 71/390 [====>.........................] - ETA: 6:05 - loss: 639.3246 - accuracy: 0.0052(128, 224, 224, 3)
(128, 511)
 72/390 [====>.........................] - ETA: 6:04 - loss: 630.5317 - accuracy: 0.0051(128, 224, 224, 3)
(128, 511)
 73/390 [====>.........................] - ETA: 6:02 - loss: 621.9800 - accuracy: 0.0050(128, 224, 224, 3)
(128, 511)
 74/390 [====>.........................] - ETA: 6:01 - loss: 613.6592 - accuracy: 0.0050(128, 224, 224, 3)
(128, 511)
 75/390 [====>.........................] - ETA: 6:00 - loss: 605.5609 - accuracy: 0.0049(128, 224, 224, 3)
(128, 511)
 76/390 [====>.........................] - ETA: 5:58 - loss: 597.6755 - accuracy: 0.0048(128, 224, 224, 3)
(128, 511)
 77/390 [====>.........................] - ETA: 5:57 - loss: 589.9946 - accuracy: 0.0049(128, 224, 224, 3)
(128, 511)
 78/390 [=====>........................] - ETA: 5:56 - l

139/390 [=========>....................] - ETA: 4:49 - loss: 329.6180 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
140/390 [=========>....................] - ETA: 4:49 - loss: 327.3082 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
141/390 [=========>....................] - ETA: 4:47 - loss: 325.0312 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
142/390 [=========>....................] - ETA: 4:47 - loss: 322.7862 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
143/390 [==========>...................] - ETA: 4:46 - loss: 320.5725 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
144/390 [==========>...................] - ETA: 4:45 - loss: 318.3898 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
145/390 [==========>...................] - ETA: 4:44 - loss: 316.2369 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
146/390 [==========>...................] - ETA: 4:43 - loss: 314.1137 - accuracy: 0.0035(128, 224, 224, 3)
(128, 511)
147/390 [==========>...................] - ETA: 4:42 - l

208/390 [===============>..............] - ETA: 3:43 - loss: 222.3512 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
209/390 [===============>..............] - ETA: 3:42 - loss: 221.3172 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
210/390 [===============>..............] - ETA: 3:41 - loss: 220.2928 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
211/390 [===============>..............] - ETA: 3:40 - loss: 219.2784 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
212/390 [===============>..............] - ETA: 3:38 - loss: 218.2735 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
213/390 [===============>..............] - ETA: 3:37 - loss: 217.2781 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
214/390 [===============>..............] - ETA: 3:36 - loss: 216.2918 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
215/390 [===============>..............] - ETA: 3:34 - loss: 215.3149 - accuracy: 0.0027(128, 224, 224, 3)
(128, 511)
216/390 [===============>..............] - ETA: 3:33 - l

277/390 [====================>.........] - ETA: 2:18 - loss: 168.5184 - accuracy: 0.0025(128, 224, 224, 3)
(128, 511)
278/390 [====================>.........] - ETA: 2:17 - loss: 167.9347 - accuracy: 0.0025(128, 224, 224, 3)
(128, 511)
279/390 [====================>.........] - ETA: 2:15 - loss: 167.3551 - accuracy: 0.0025(128, 224, 224, 3)
(128, 511)
280/390 [====================>.........] - ETA: 2:14 - loss: 166.7797 - accuracy: 0.0025(128, 224, 224, 3)
(128, 511)
281/390 [====================>.........] - ETA: 2:13 - loss: 166.2084 - accuracy: 0.0024(128, 224, 224, 3)
(128, 511)
282/390 [====================>.........] - ETA: 2:12 - loss: 165.6411 - accuracy: 0.0024(128, 224, 224, 3)
(128, 511)
283/390 [====================>.........] - ETA: 2:10 - loss: 165.0779 - accuracy: 0.0024(128, 224, 224, 3)
(128, 511)
284/390 [====================>.........] - ETA: 2:09 - loss: 164.5186 - accuracy: 0.0024(128, 224, 224, 3)
(128, 511)
285/390 [====================>.........] - ETA: 2:08 - l

346/390 [=========================>....] - ETA: 53s - loss: 136.1562 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
347/390 [=========================>....] - ETA: 52s - loss: 135.7818 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
348/390 [=========================>....] - ETA: 50s - loss: 135.4096 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
349/390 [=========================>....] - ETA: 49s - loss: 135.0395 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
350/390 [=========================>....] - ETA: 48s - loss: 134.6715 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
351/390 [==========================>...] - ETA: 47s - loss: 134.3056 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
352/390 [==========================>...] - ETA: 46s - loss: 133.9418 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
353/390 [==========================>...] - ETA: 44s - loss: 133.5800 - accuracy: 0.0023(128, 224, 224, 3)
(128, 511)
354/390 [==========================>...] - ETA: 43s - loss: 133.